In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import sys
# sys.path.append("../../")

sys.path.append("../")
# sys.path.append("..")
from early_exit.patching.method_patching import replace_attention_layers, set_transformer_early_exit_mode
from shared_utils.generate import format_conversation, transform_conversations
from early_exit.util import module_name_is_layer_base
import numpy as np
from early_exit.util import get_model
from shared_utils.load import get_tokenizer, configs_from_yaml
from shared_utils.generate import generate_text
import random
from early_exit_teacher.visualization import visualize_tokens_by_exit_layer, create_html_visualization
from IPython.display import HTML, display
from early_exit.util import module_name_is_layer_base
torch.set_grad_enabled(False)
print("Disabled automatic differentiation")
import torch.nn.functional as F
import pandas as pd

Disabled automatic differentiation


## Setup

In [9]:
def compare_with_kv_caching(set_early_exit_layer, show_visualization = True, show_df = True):
    set_transformer_early_exit_mode(model, 'off')
    generated_tokens = []
    max_new_tokens = 10
    generated = inputs["input_ids"]
    chosen_exit_layers = []
    all_student_logits = []
    all_teacher_logits = []
    for step in range(max_new_tokens):  # generate 10 tokens
        if step == 0:
            student_outputs = model(generated, use_cache=True)
            student_logits = student_outputs.logits
            student_cache = student_outputs.past_key_values
            teacher_outputs = model(generated, use_cache=True)
            teacher_cache = teacher_outputs.past_key_values
            teacher_logits = teacher_outputs.logits
        else:
            # Pass only the new token and the cache
            student_outputs = model(
                next_token,
                past_key_values=student_cache,
                use_cache=True,
                output_hidden_states=True
            )
            early_exit_layer = set_early_exit_layer(step)
            hidden_states = student_outputs.hidden_states
            hidden_states = torch.stack(student_outputs.hidden_states)[1:]
            student_logits = model.lm_head(hidden_states[early_exit_layer])
            all_student_logits.append(student_logits)
            student_cache = student_outputs.past_key_values  # updated with new token
            # print(len(past_key_values))
            for layer in range(early_exit_layer, 28):
                student_cache[layer][0][:, :, -1] = student_cache[early_exit_layer][0][:, :, -1] # keys
                student_cache[layer][1][:, :, -1] = student_cache[early_exit_layer][1][:, :, -1] # values
            # print(len(student_cache), student_cache[0][0].shape)
            
            teacher_outputs = model(
                next_token,
                past_key_values=teacher_cache,
                use_cache=True
            )
            teacher_cache = teacher_outputs.past_key_values
            teacher_logits = teacher_outputs.logits
            all_teacher_logits.append(teacher_logits)
        # Take the most likely next token (greedy decoding here)
        # next_token = torch.argmax(student_logits[:, -1, :], dim=-1).unsqueeze(-1)
        next_token = torch.argmax(teacher_logits[:, -1, :], dim=-1).unsqueeze(-1)
        if step > 0: 
            generated_tokens.append(next_token.item())
            chosen_exit_layers.append(early_exit_layer)
        else:
            generated_tokens.append(next_token.item())
            chosen_exit_layers.append(-1)
        # Append to generated sequence
        generated = torch.cat([generated, next_token], dim=-1)
    
    # print(tokenizer.decode(generated[0]))
    all_student_logits = torch.concatenate(all_student_logits, axis = 0).transpose(0,1)
    all_teacher_logits = torch.concatenate(all_teacher_logits, axis = 0).transpose(0,1)
    # all_student_logits.shape
    tokens = [tokenizer.decode([token]) for token in generated_tokens]
    layers = [27 if item == 27 or item == -1 else item for item in chosen_exit_layers]
    early_exit_layers = early_exit_layer_idxs.tolist()  # Convert tensor to list if needed
    # Display the visualization
    if show_visualization: display(visualize_tokens_by_exit_layer(tokens, layers, early_exit_layers, 
                                         title="Committed Early Exit Token Generation"))
    
    chosen_exit_layers_tensor = torch.tensor(chosen_exit_layers[1:], device=device).unsqueeze(0).float()
    chosen_exit_layers_tensor = torch.where(
                        chosen_exit_layers_tensor == 27,
                        torch.full_like(chosen_exit_layers_tensor, float('inf')),
                        chosen_exit_layers_tensor
                    )
    repeated_sft_teacher_generated_tokens = generated[:, :-1]
    
    set_transformer_early_exit_mode(model, 'sft_student')
    sft_student_output_scores, collected_exit_logits = model(repeated_sft_teacher_generated_tokens,\
                                                                 prescribed_exit_layer_idxs=chosen_exit_layers_tensor)
    
    sft_student_output = sft_student_output_scores.logits.squeeze()[20:]
    kv_cache_output = all_student_logits.squeeze()
    
    student_probs = F.softmax(sft_student_output, dim=-1)
    teacher_cache_probs = F.softmax(all_teacher_logits.squeeze(), dim=-1)
    student_cache_probs = F.softmax(kv_cache_output, dim=-1)
    
    # model_outputs = model(repeated_sft_teacher_generated_tokens)
    # model_probs = F.softmax(model_outputs.logits[0, 20:], dim = -1)
    
    # set_transformer_early_exit_mode(frozen_model, 'off')
    # off_outputs = frozen_model(repeated_sft_teacher_generated_tokens)
    # off_probs = F.softmax(off_outputs.logits[0, 20:], dim = -1)
    
    pd.options.display.float_format = "{:.2f}".format
    rows = []
    
    def get_prob_token(probs):
        top_id = torch.argmax(probs).item()
        top_prob = probs[top_id].item()
        top_token = tokenizer.decode([top_id])
        return top_prob, top_token
    
    for idx in range(len(student_probs)):
        # Student
        student_top_prob, student_top_token = get_prob_token(student_probs[idx])
        # teacher_top_prob, teacher_top_token = get_prob_token(teacher_probs[idx])
        
        student_cache_top_prob, student_cache_top_token = get_prob_token(student_cache_probs[idx])
        teacher_cache_top_prob, teacher_cache_top_token = get_prob_token(teacher_cache_probs[idx])
    
        
        # model_top_prob, model_top_token = get_prob_token(model_probs[idx])
        
        # off_top_prob, off_top_token = get_prob_token(off_probs[idx])
    
        rows.append({
            # "Position": idx,
            "Student Token": student_top_token,
            "Student Prob": student_top_prob,
            # "Teacher Token": teacher_top_token,
            # "Teacher Prob": teacher_top_prob,
            "Student Cache Token": student_cache_top_token,
            "Student Cache Prob": student_cache_top_prob,
            "Teacher Cache Token": teacher_cache_top_token,
            "Teacher Cache Prob": teacher_cache_top_prob,
            # "Model Token": model_top_token,
            # "Model Prob": model_top_prob,
            # "Off Token": off_top_token,
            # "Off Prob": off_top_prob
        })
    
    df = pd.DataFrame(rows)
    if show_df: display(df)


In [3]:
# Model configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

print(f"Loading model: {model_name}")
print(f"Device: {device}")

tokenizer = get_tokenizer(model_name)
model_config_path = "/project/project_465001340/fair_stuff/externalization/config_deepseek.yaml"   # args.model_config_path
config = configs_from_yaml(model_config_path, tokenizer.eos_token_id)

model = get_model(model_name, config['model'], device)
model = replace_attention_layers(model, config['lora'], device)
# set_transformer_early_exit_mode(model, 'off')

# Load tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    

config['generation']['max_new_tokens'] = 10

print(f"Tokenizer loaded. Vocab size: {tokenizer.vocab_size}")
print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")



prompt = "Explain the concept of recursion in programming."
system_prompt = "You are a helpful programming tutor."
prefiller = ""

pre_transformed_conversation = format_conversation(user_prompts = [prompt], system_prompt=system_prompt)
formatted_prompt = transform_conversations(pre_transformed_conversation, prefiller)[0]
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
input_ids = inputs.input_ids

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
Device: cuda
address this hack!
g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

trainable params: 2,179,072 || all params: 1,779,276,294 || trainable%: 0.1225
Tokenizer loaded. Vocab size: 151643
EOS token: <｜end▁of▁sentence｜> (ID: 151643)
transform_conversations currently only for Deepseek models!


In [4]:
from early_exit.util import module_name_is_layer_base
early_exit_layer_idxs = []
for name, module in model.named_modules():
    if module_name_is_layer_base(name):
        layer_idx = int(name.split('.')[-1])
        early_exit_layer_idxs.append(layer_idx)

early_exit_layer_idxs = torch.tensor(early_exit_layer_idxs, dtype = torch.int32)  # Add inf for final layer

## Analysis

In [14]:
def set_early_exit_layer(step):
    return 27
compare_with_kv_caching(set_early_exit_layer, show_visualization=False)

,Student Token,Student Prob,Student Cache Token,Student Cache Prob,Teacher Cache Token,Teacher Cache Prob
0,",",1.00,",",1.00,",",1.00
1,so,0.78,so,0.78,so,0.78
2,I,0.93,I,0.93,I,0.93
3,need,0.65,need,0.65,need,0.65
4,to,1.00,to,1.00,to,1.00
5,explain,0.75,explain,0.75,explain,0.75
6,recursion,0.50,recursion,0.50,recursion,0.50
7,in,0.98,in,0.98,in,0.98
8,programming,1.00,programming,1.00,programming,1.00


In [16]:
def set_early_exit_layer(step):
    if step == 2: return 25
    return 27
compare_with_kv_caching(set_early_exit_layer, show_visualization=True)

,Student Token,Student Prob,Student Cache Token,Student Cache Prob,Teacher Cache Token,Teacher Cache Prob
0,",",1.00,",",1.00,",",1.00
1,so,0.99,so,1.00,so,0.78
2,Helvetica,0.01,I,0.92,I,0.93
3,一定程度,0.01,need,0.64,need,0.65
4,transparent,0.03,to,1.00,to,1.00
5,lay,0.01,explain,0.74,explain,0.75
6,enses,0.01,recursion,0.50,recursion,0.50
7,Helvetica,0.00,in,0.98,in,0.98
8,incoming,0.06,programming,1.00,programming,1.00


### Some fishy behaviour in the hidden states when exit is set to inf

In [19]:
set_transformer_early_exit_mode(model, 'sft_teacher')
sft_teacher_response, (sft_teacher_generated_tokens, 
                          sft_teacher_final_layer_logprobs, 
                          gathered_early_exit_hidden_states) = generate_text(
        model=model,
        prompt=prompt,
        system_prompt=system_prompt,
        prefiller=prefiller,
        tokenizer=tokenizer,
        generation_config=config['generation'],
        device=device
    )
repeated_sft_teacher_generated_tokens = sft_teacher_generated_tokens[:, :-1] # Removing the last token to match the log probs and hidden states shape
chosen_exit_layers_tensor = torch.inf * torch.ones([1,9])
set_transformer_early_exit_mode(model, 'off')
model_outputs = model(repeated_sft_teacher_generated_tokens, output_hidden_states = True)
set_transformer_early_exit_mode(model, 'sft_student')
student_outputs = model(repeated_sft_teacher_generated_tokens, prescribed_exit_layer_idxs = chosen_exit_layers_tensor, output_hidden_states = True)

transform_conversations currently only for Deepseek models!
full_tokenize currently only for Deepseek models!
prompt tokens shape: torch.Size([1, 20])


In [41]:
def compare(T1,T2):
    if (T1 == T2).all().item():
        print("Input tensors are equal")
    else:
        print("Input tensors are NOT equal")
        print(f"Tensor 1 shape = {T1.shape}")
        print(f"Tensor 1 sum = {T1.sum().item():.2f}")
        print(f"Tensor 2 sum = {T2.sum().item():.2f}")

In [42]:
compare(student_outputs[0].logits, model_outputs.logits)

Input tensors are equal


In [43]:
layer_idx = -1
compare(student_outputs[0].hidden_states[layer_idx], model_outputs.hidden_states[layer_idx])

Input tensors are equal


In [44]:
layer_idx = -2
compare(student_outputs[0].hidden_states[layer_idx], model_outputs.hidden_states[layer_idx])

Input tensors are NOT equal
Tensor 1 shape = torch.Size([1, 29, 1536])
Tensor 1 sum = 71.16
Tensor 2 sum = 1184.38


In [45]:
layer_idx = 0
compare(student_outputs[0].hidden_states[layer_idx], model_outputs.hidden_states[layer_idx])

Input tensors are NOT equal
Tensor 1 shape = torch.Size([1, 29, 1536])
Tensor 1 sum = 144.88
Tensor 2 sum = 11.01
